In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("google/yamnet/tensorFlow2/yamnet")

# print("Path to model files:", path)

In [2]:
import utils as u
import tensorflow_hub as hub

yamnet = hub.load('/Users/zhaokaiyang/.cache/kagglehub/models/google/yamnet/tensorFlow2/yamnet/1')

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-04-27 14:18:30.762906: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-27 14:18:30.763001: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# 指定你的音频文件夹路径
folder_path = '/Users/zhaokaiyang/Downloads/PMEmo2019/chorus/'
# 处理文件夹并获取 embeddings
embeddings_dict = u.process_folder(folder_path, yamnet)

2024-04-27 14:18:52.065859: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-27 14:18:52.073811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [7]:
# 打印所有键
print(list(embeddings_dict.keys()))

['752.mp3', '746.mp3', '791.mp3', '550.mp3', '236.mp3', '222.mp3', '544.mp3', '578.mp3', '587.mp3', '424.mp3', '342.mp3', '356.mp3', '418.mp3', '381.mp3', '395.mp3', '140.mp3', '626.mp3', '632.mp3', '96.mp3', '168.mp3', '829.mp3', '69.mp3', '197.mp3', '801.mp3', '815.mp3', '55.mp3', '7.mp3', '54.mp3', '814.mp3', '6.mp3', '40.mp3', '68.mp3', '828.mp3', '196.mp3', '97.mp3', '169.mp3', '83.mp3', '141.mp3', '627.mp3', '394.mp3', '419.mp3', '431.mp3', '425.mp3', '343.mp3', '592.mp3', '579.mp3', '551.mp3', '237.mp3', '974.mp3', '784.mp3', '948.mp3', '790.mp3', '747.mp3', '753.mp3', '745.mp3', '989.mp3', '751.mp3', '779.mp3', '786.mp3', '792.mp3', '962.mp3', '221.mp3', '235.mp3', '209.mp3', '584.mp3', '590.mp3', '433.mp3', '355.mp3', '341.mp3', '427.mp3', '396.mp3', '382.mp3', '157.mp3', '631.mp3', '625.mp3', '143.mp3', '95.mp3', '81.mp3', '180.mp3', '4.mp3', '816.mp3', '56.mp3', '42.mp3', '43.mp3', '803.mp3', '5.mp3', '57.mp3', '181.mp3', '80.mp3', '94.mp3', '624.mp3', '142.mp3', '156.mp3', 

In [8]:
# 创建一个新字典，其键是转换后的格式
new_embeddings_dict = {}
for key in embeddings_dict.keys():
    new_key = key[:-4] + ".0"  # 移除 '.mp3' 并添加 '.0'
    new_embeddings_dict[new_key] = embeddings_dict[key]

embeddings_dict = new_embeddings_dict


In [9]:
# 打印所有键
print(list(embeddings_dict.keys()))

['752.0', '746.0', '791.0', '550.0', '236.0', '222.0', '544.0', '578.0', '587.0', '424.0', '342.0', '356.0', '418.0', '381.0', '395.0', '140.0', '626.0', '632.0', '96.0', '168.0', '829.0', '69.0', '197.0', '801.0', '815.0', '55.0', '7.0', '54.0', '814.0', '6.0', '40.0', '68.0', '828.0', '196.0', '97.0', '169.0', '83.0', '141.0', '627.0', '394.0', '419.0', '431.0', '425.0', '343.0', '592.0', '579.0', '551.0', '237.0', '974.0', '784.0', '948.0', '790.0', '747.0', '753.0', '745.0', '989.0', '751.0', '779.0', '786.0', '792.0', '962.0', '221.0', '235.0', '209.0', '584.0', '590.0', '433.0', '355.0', '341.0', '427.0', '396.0', '382.0', '157.0', '631.0', '625.0', '143.0', '95.0', '81.0', '180.0', '4.0', '816.0', '56.0', '42.0', '43.0', '803.0', '5.0', '57.0', '181.0', '80.0', '94.0', '624.0', '142.0', '156.0', '630.0', '383.0', '397.0', '368.0', '426.0', '432.0', '354.0', '591.0', '585.0', '234.0', '552.0', '546.0', '220.0', '793.0', '778.0', '750.0', '744.0', '740.0', '754.0', '967.0', '973.0

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 加载 Arousal & Valence 数据
av_data = pd.read_csv('/Users/zhaokaiyang/Downloads/PMEmo2019/annotations/static_annotations.csv')  # 确保路径正确

# 初始化数据列表
X = []
y = []

# 遍历数据行
for index, row in av_data.iterrows():
    music_id = str(row['musicId'])  # 根据需要将 musicId 转换为字符串，假设 embeddings_dict 的键是字符串形式的 musicId
    if music_id in embeddings_dict:
        X.append(embeddings_dict[music_id])
        y.append([row['Arousal(mean)'], row['Valence(mean)']])
    else:
        print(f"Music ID {music_id} not found in embeddings_dict")  # 输出未找到的音乐 ID

# 将列表转换为 NumPy 数组
X = np.array(X)
y = np.array(y)

print(f"Total samples loaded: {len(X)}")  # 输出加载的样本总数

if len(X) > 0:
    # 划分数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
else:
    print("No data to split. Check your dataset and embeddings.")


Total samples loaded: 767
Training set size: 613, Test set size: 154


In [11]:
from models import build_model

# 根据输入数据的形状创建模型
model = build_model(input_shape=(60, 1024))  # 确保这与你的 embeddings 形状匹配



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on

In [12]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-04-27 14:22:52.446739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - ETA: 0s - loss: 0.6370 - mae: 0.4765WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x17e344dc0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-04-27 14:22:56.406311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 6s 173ms/step - loss: 0.6370 - mae: 0.4765 - val_loss: 0.0829 - val_mae: 0.2239
Epoch 2/20
20/20 [==============================] - 1s 64ms/step - loss: 0.1170 - mae: 0.1932 - val_loss: 0.0470 - val_mae: 0.1695
Epoch 3/20
20/20 [==============================] - 1s 63ms/step - loss: 0.1910 - mae: 0.1615 - val_loss: 0.0457 - val_mae: 0.1563
Epoch 4/20
20/20 [==============================] - 1s 67ms/step - loss: 0.1443 - mae: 0.1395 - val_loss: 0.0280 - val_mae: 0.1291
Epoch 5/20
20/20 [==============================] - 1s 67ms/step - loss: 0.0991 - mae: 0.1253 - val_loss: 0.0288 - val_mae: 0.1257
Epoch 6/20
20/20 [==============================] - 1s 64ms/step - loss: 0.1286 - mae: 0.1144 - val_loss: 0.0249 - val_mae: 0.1216
Epoch 7/20
20/20 [==============================] - 1s 64ms/step - loss: 0.0179 - mae: 0.0897 - val_loss: 0.0240 - val_mae: 0.1166
Epoch 8/20
20/20 [==============================] - 1s 64ms/step - loss: 0.0192 - mae: 0.0818

In [13]:
# 评估模型性能
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


5/5 [==============================] - 0s 27ms/step - loss: 0.0235 - mae: 0.1178
Test Loss: 0.023481957614421844, Test MAE: 0.1178331971168518


In [14]:
model.save('/Users/zhaokaiyang/Documents/24Spring/Deep Learning/Final Project/emotionPredict.h5')  # 保存为 SavedModel 格式

In [15]:
from tensorflow.keras.models import load_model
from tcn import TCN  # 确保正确导入 TCN 层的定义

# 加载模型时指定自定义层
model_test = load_model('/Users/zhaokaiyang/Documents/24Spring/Deep Learning/Final Project/emotionPredict.h5', custom_objects={'TCN': TCN})

In [16]:
import librosa

wave, sr = librosa.load("/Users/zhaokaiyang/Documents/24Spring/Deep Learning/Final Project/test.mp3", sr=16000, mono=True)

In [18]:
feature_test = u.extract_features(wave, yamnet)
feature_test = np.expand_dims(feature_test, axis=0)

In [19]:
feature_test.shape

(1, 60, 1024)

In [20]:
predicted_values = model_test.predict(feature_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2024-04-27 14:23:52.173725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 539ms/step


In [21]:
arousal_test, valence_test = predicted_values[0][0], predicted_values[0][1]

In [22]:
print(arousal_test, valence_test)

0.629775 0.36699024
